In [1]:
!pip -q install pdfplumber nltk pandas numpy openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 125.9 MB/s eta 0:00:00


In [2]:
!pip -q install openai pdfplumber pandas

from pathlib import Path
import pdfplumber, re, json, os
import pandas as pd
from openai import OpenAI
from google.colab import userdata

# 🔑 Load your OpenAI key correctly (secret name = "sandra")
api_key = userdata.get("sandra")
client = OpenAI(api_key=api_key)

PDF_DIR = Path("/content/pdfs")
PDF_DIR.mkdir(exist_ok=True, parents=True)
for p in Path("/content").glob("*.pdf"):
    shutil.move(str(p), str(PDF_DIR / p.name))

print("Now in /content/pdfs:", [p.name for p in PDF_DIR.glob("*.pdf")])

Now in /content/pdfs: []


In [3]:
# === Stage 2: Page-Aware Text Extraction (safer, no re-split) ===
import re, pdfplumber
from pathlib import Path

def extract_clean_text_pages(pdf_path: Path):
    """
    Return: list[(page_num:int, text:str)]
    - Cleans hyphenated line breaks & excess newlines
    - Drops Abstract only on the first few pages
    - Stops at the first page that *starts with* a References/Bibliography heading
    - No global join/re-split (so no content loss)
    """
    pages = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            t = page.extract_text() or ""
            # de-hyphenate across line breaks
            t = re.sub(r"(\w)-\n(\w)", r"\1\2", t)
            # normalize whitespace
            t = re.sub(r"[ \t]+", " ", t)
            t = re.sub(r"\n{2,}", "\n", t).strip()
            pages.append((i, t))

    # Remove Abstract only within the first 3 pages (if present)
    for k in range(min(3, len(pages))):
        pnum, txt = pages[k]
        # remove a leading Abstract block conservatively
        txt2 = re.sub(
            r"(?is)^\s*abstract\b.*?(?=\n[A-Z][^\n]{0,80}\n|\Z)",  # up to next likely heading or end
            "",
            txt,
            count=1
        ).strip()
        pages[k] = (pnum, txt2)

    # Find the first page that LOOKS like a references page and stop there.
    def is_refs_page(txt: str) -> bool:
        # only count if heading near the top of the page
        head = "\n".join(txt.splitlines()[:20])
        return bool(re.search(r"(?im)^\s*(references|bibliograph\w*)\s*$", head))

    stop_at = None
    for idx, (pnum, txt) in enumerate(pages):
        if is_refs_page(txt):
            stop_at = idx
            break
    if stop_at is not None:
        pages = pages[:stop_at]  # cut at the references page, keep original page splits

    return pages

In [ ]:
# Collect pages for each PDF (no merging)
PDF_DIR = Path("/content/pdfs")
pdf_paths = sorted(PDF_DIR.glob("*.pdf"))
assert pdf_paths, f"No PDFs found in {PDF_DIR}. Upload PDFs first."

all_docs_pages = {}  # {pdf_name: [(page_num, text), ...]}
for pdf_path in pdf_paths:
    pages = extract_clean_text_pages(pdf_path)
    all_docs_pages[pdf_path.name] = pages
    print(f"✅ {pdf_path.name}: {len(pages)} pages kept (pre-LLM)")

✅ 2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf: 78 pages kept (pre-LLM)
✅ 2008_MATABANE_FE3.pdf: 44 pages kept (pre-LLM)
✅ 2009_Bontle Nkuna_0605886P_Honours Report.pdf: 50 pages kept (pre-LLM)
✅ 2011_Peters_East Markoye_2011.pdf: 62 pages kept (pre-LLM)


In [5]:
# === Stage 3: Page-Referenced AI Extraction (per-PDF; stricter) ===
from tqdm import tqdm
import json, textwrap
from openai import OpenAI


schema = {
  "page_number": "integer",
  "metadata": {
    "title": "string",
    "author": "string",
    "year": "integer",
    "supervisor": "string",
    "institution": "string",
    "location": "string"
  },
  "geology": {
    "region": "string",
    "formation": "string",
    "rock_types": ["list of strings"],
    "minerals": ["list of strings"],
    "structures": ["list of strings"],
    "tectonic_setting": "string"
  },
  "geochronology": {
    "sample_id": "string",
    "method": "string",
    "age_Ma": "float",
    "error_Ma": "float",
    "rock_unit": "string",
    "evidence": "string"
  },
  "geochemistry": {
    "sample_id": "string",
    "analyte": "string",
    "value": "float",
    "unit": "string",
    "method": "string",
    "context": "string"
  },
  "metallogeny": {
    "mineralisation_type": "string",
    "associated_structures": ["list of strings"],
    "host_rocks": ["list of strings"],
    "ore_minerals": ["list of strings"],
    "alteration": "string"
  }
}

def ask_model(page_num: int, page_text: str, desc="Extracting"):
    # keep per-page cap; most pages are << 5500 chars anyway
    messages = [
        {"role": "system", "content":
         "You are a geology data extraction AI. Output valid JSON ONLY."},
        {"role": "user", "content": f"""
Extract ONLY from the page text given. Obey this schema exactly:
{json.dumps(schema, indent=2)}

Rules:
- Set "page_number": {page_num}.
- Use ONLY facts present in the page text. Do NOT infer, guess, or use outside knowledge.
- If a field is not present, set empty string (""), 0, null, or [] as appropriate (NO 'Unknown').
- If nothing relevant is present, return [] (empty list).
- Keep lists deduplicated.

PAGE TEXT (page {page_num}):
{text[:5500]}
"""}
    ]

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.1,
        max_tokens=1200,
        response_format={"type": "json_object"}  # force JSON
    )
    raw = resp.choices[0].message.content.strip()
    try:
        data = json.loads(raw)
    except Exception:
        # try to salvage JSON object from code fences if present
        raw2 = raw.strip("`").replace("json", "").strip()
        data = json.loads(raw2)
    return data

# Run per PDF → each gets its own extracted JSON
OUT_DIR = Path("/content/extracted_json"); OUT_DIR.mkdir(parents=True, exist_ok=True)

for pdf_name, page_list in all_docs_pages.items():
    out_path = OUT_DIR / f"{Path(pdf_name).stem}.extracted.json"
    records = []
    for pnum, text in tqdm(page_list, desc=f"Extracting {pdf_name}", leave=False):
        if not text:
            continue
        try:
            rec = ask_model(pnum, text)
            if rec:
                records.append(rec)
        except Exception as e:
            print(f"⚠️ {pdf_name} p{pnum}: {e}")

    with out_path.open("w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)
    print(f"✅ Saved {len(records)} page records → {out_path}")

✅ Saved 78 page records → /content/extracted_json/2007_Tshibubudze_THE MARKOYE FAULT_2007.extracted.json


✅ Saved 44 page records → /content/extracted_json/2008_MATABANE_FE3.extracted.json


✅ Saved 50 page records → /content/extracted_json/2009_Bontle Nkuna_0605886P_Honours Report.extracted.json


✅ Saved 62 page records → /content/extracted_json/2011_Peters_East Markoye_2011.extracted.json


✅ Saved 186 page records → /content/extracted_json/2021_Volcanic Architecture of the Hounde and Boromo Greenstone Belts_PhD_Thesis.extracted.json


In [6]:
# === Stage 4: Reference-Aware Flatten (per-PDF + combined) ===
from pathlib import Path
import json, pandas as pd
from collections import defaultdict

IN_DIR  = Path("/content/extracted_json")       # outputs of Stage 3
OUT_DIR = Path("/content/flattened_csv"); OUT_DIR.mkdir(parents=True, exist_ok=True)

def normalize_records(raw):
    """Stage-3 may store a list of dicts, or a dict per page. Normalize to list[dict]."""
    if not raw:
        return []
    out = []
    for item in raw:
        if isinstance(item, list):
            out.extend([x for x in item if isinstance(x, dict)])
        elif isinstance(item, dict):
            out.append(item)
    return out

def flatten_one_pdf(pdf_name: str, recs: list[dict]) -> pd.DataFrame:
    """
    Flatten per-PDF:
      - de-dup (section, field, value) within this PDF
      - union the page numbers where it appears
    """
    combined = defaultdict(lambda: {
        "pdf_name": pdf_name, "section": None, "field": None, "value": None, "pages": set()
    })

    for rec in recs:
        page = rec.get("page_number", None)
        for section, fields in rec.items():
            if section == "page_number":
                continue

            # section: metadata, geology, geochronology, geochemistry, metallogeny
            if isinstance(fields, dict):
                for k, v in fields.items():
                    # keep zeros and False; skip only None or empty strings
                    if v is None or (isinstance(v, str) and v.strip() == ""):
                        continue
                    if isinstance(v, list):
                        for vv in v:
                            if vv is None or (isinstance(vv, str) and vv.strip() == ""):
                                continue
                            key = (section, k, str(vv))
                            node = combined[key]
                            node["section"] = section; node["field"] = k; node["value"] = vv
                            if page is not None: node["pages"].add(str(page))
                    else:
                        key = (section, k, str(v))
                        node = combined[key]
                        node["section"] = section; node["field"] = k; node["value"] = v
                        if page is not None: node["pages"].add(str(page))

            elif isinstance(fields, list):
                for item in fields:
                    if item is None or (isinstance(item, str) and item.strip() == ""):
                        continue
                    key = (section, "list_item", str(item))
                    node = combined[key]
                    node["section"] = section; node["field"] = "list_item"; node["value"] = item
                    if page is not None: node["pages"].add(str(page))

    flat = [{
        "pdf_name": info["pdf_name"],
        "section": info["section"],
        "field": info["field"],
        "value": info["value"],
        "pages": ", ".join(sorted(info["pages"])) if info["pages"] else ""
    } for info in combined.values()]

    return pd.DataFrame(flat, columns=["pdf_name","section","field","value","pages"])

# ---- Run over all per-PDF JSONs from Stage 3 ----
json_paths = sorted(IN_DIR.glob("*.extracted.json"))
assert json_paths, f"No Stage-3 JSONs found in {IN_DIR}. Run Stage 3 first."

dfs = []
for jp in json_paths:
    # derive a friendly pdf name (e.g., "Thesis.extracted.json" -> "Thesis.pdf")
    pdf_name = jp.stem.replace(".extracted", "") + ".pdf"
    raw = json.loads(jp.read_text(encoding="utf-8"))
    recs = normalize_records(raw)
    df   = flatten_one_pdf(pdf_name, recs)

    out_csv = OUT_DIR / f"{jp.stem}.flatten.csv"
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"✓ {pdf_name}: {len(df)} rows → {out_csv}")
    dfs.append(df)

# combined CSV
combined_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(columns=["pdf_name","section","field","value","pages"])
combined_df.to_csv(OUT_DIR / "_combined_flatten.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ Combined flatten saved → {OUT_DIR / '_combined_flatten.csv'}")

# quick peek
combined_df.head(10)


✓ 2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf: 417 rows → /content/flattened_csv/2007_Tshibubudze_THE MARKOYE FAULT_2007.extracted.flatten.csv
✓ 2008_MATABANE_FE3.pdf: 276 rows → /content/flattened_csv/2008_MATABANE_FE3.extracted.flatten.csv
✓ 2009_Bontle Nkuna_0605886P_Honours Report.pdf: 306 rows → /content/flattened_csv/2009_Bontle Nkuna_0605886P_Honours Report.extracted.flatten.csv
✓ 2011_Peters_East Markoye_2011.pdf: 278 rows → /content/flattened_csv/2011_Peters_East Markoye_2011.extracted.flatten.csv
✓ 2021_Volcanic Architecture of the Hounde and Boromo Greenstone Belts_PhD_Thesis.pdf: 798 rows → /content/flattened_csv/2021_Volcanic Architecture of the Hounde and Boromo Greenstone Belts_PhD_Thesis.extracted.flatten.csv

✅ Combined flatten saved → /content/flattened_csv/_combined_flatten.csv


,pdf_name,section,field,value,pages
0,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,title,RELATIVE TIMING OF STRUCTURAL EVENTS: THE MARK...,1
1,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,author,ASINNE TSHIBUBUDZE,1
2,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,year,2007,"1, 2"
3,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,supervisor,Prof. Kim A.A Hein,1
4,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,institution,University of the Witwatersrand,"1, 2"
5,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,location,"Johannesburg, South Africa",1
6,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,geochronology,age_Ma,0,"1, 11, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, ..."
7,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,geochronology,error_Ma,0,"1, 10, 11, 13, 14, 15, 16, 17, 18, 19, 2, 20, ..."
8,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,geochemistry,value,0,"1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, ..."
9,2007_Tshibubudze_THE MARKOYE FAULT_2007.pdf,metadata,location,Johannesburg,2
